## Step 2: Data Exploration:

In [4]:
from google.colab import files
uploaded = files.upload()


Saving Egypt_Data_Split.zip to Egypt_Data_Split.zip


In [5]:
import zipfile
import os

zip_path = "Egypt_Data_Split.zip"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(".")

print("Extracted Successfully!")


Extracted Successfully!


In [6]:
import os
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import seaborn as sns
import cv2
from collections import Counter

In [19]:
dataset_path = "Egypt_Data_Split/train"


In [20]:
# List all land type categories
categories = os.listdir(dataset_path)

## Step 3: Data Preprocessing

In [21]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
# Resize, normalize, and convert to RGB
def load_and_preprocess_images(path, size=(64, 64)):
    data, labels = [], []
    for category in categories:
        category_path = os.path.join(path, category)
        for img_name in os.listdir(category_path):
            img = cv2.imread(os.path.join(category_path, img_name))
            img = cv2.resize(img, size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            img = img / 255.0  # Normalize to [0,1]
            data.append(img)
            labels.append(category)
    return np.array(data, dtype="float32"), np.array(labels)

In [22]:
def load_and_preprocess_images(path, size=(128,128)):
    data = []
    labels = []

    for category in os.listdir(path):
        category_path = os.path.join(path, category)

        # Skip non-folder items
        if not os.path.isdir(category_path):
            continue

        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)

            # Attempt to read image
            img = cv2.imread(img_path)

            # If image can't be read → skip it
            if img is None:
                print("Skipped unreadable file:", img_path)
                continue

            img = cv2.resize(img, size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img / 255.0

            data.append(img)
            labels.append(category)

    return np.array(data), np.array(labels)


In [23]:
def load_and_preprocess_images(path, size=(128,128)):
    data = []
    labels = []

    # Loop over each class folder
    for category in os.listdir(path):
        category_path = os.path.join(path, category)

        # تأكد إنه فولدر مش فايل
        if not os.path.isdir(category_path):
            continue

        # Loop over images inside class folder
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)

            # Read image
            img = cv2.imread(img_path)

            # Skip unreadable or non-image files
            if img is None:
                # print("Skipped unreadable file:", img_path)
                continue

            img = cv2.resize(img, size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img / 255.0

            data.append(img)
            labels.append(category)

    return np.array(data), np.array(labels)


In [24]:
# Load data
data, labels = load_and_preprocess_images(dataset_path, size=(128, 128))

In [25]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [17]:
# Encode labels into integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)  # Convert category names to integer labels

In [27]:
len(labels)
set(labels)


{np.str_('Crop Data'), np.str_('Desert Data'), np.str_('Urban Data')}

In [28]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

print("Encoded labels example:", labels_encoded[:10])
print("Classes:", label_encoder.classes_)


Encoded labels example: [1 1 1 1 1 1 1 1 1 1]
Classes: ['Crop Data' 'Desert Data' 'Urban Data']


In [29]:
from tensorflow.keras.utils import to_categorical

labels_categorical = to_categorical(labels_encoded, num_classes=len(label_encoder.classes_))

print("One-hot shape:", labels_categorical.shape)


One-hot shape: (1113, 3)


In [30]:
# Convert to one-hot encoding
labels_categorical = to_categorical(labels_encoded)

In [31]:
from sklearn.model_selection import train_test_split
import numpy as np

# تأكد إن X و y لهم نفس العدد
print("Data:", data.shape)
print("Labels:", labels_categorical.shape)

# Shuffle قبل التقسيم
indices = np.arange(len(data))
np.random.shuffle(indices)

data_shuffled = data[indices]
labels_shuffled = labels_categorical[indices]

# تقسيم 70% Train و 30% Temp
X_train, X_temp, y_train, y_temp = train_test_split(
    data_shuffled, labels_shuffled,
    test_size=0.3,
    random_state=42
)

# تقسيم الـ Temp إلى 20% Val و 10% Test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=1/3,
    random_state=42
)

# طباعة أعداد العينات للتأكد
print("Train samples:", len(X_train))
print("Validation samples:", len(X_val))
print("Test samples:", len(X_test))


Data: (1113, 128, 128, 3)
Labels: (1113, 3)
Train samples: 779
Validation samples: 222
Test samples: 112


In [32]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([

    # اول طبقة Convolution
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),

    # الطبقة التانية
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # الطبقة التالتة
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Flatten
    layers.Flatten(),

    # Fully connected
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),

    # Output layer (3 classes)
    layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,027 (12.61 MB)

 Trainable params: 3,305,027 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [33]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32
)


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.5258 - loss: 0.9046 - val_accuracy: 0.7072 - val_loss: 0.6219
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.7898 - loss: 0.4913 - val_accuracy: 0.9324 - val_loss: 0.1835
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.9083 - loss: 0.2893 - val_accuracy: 0.9189 - val_loss: 0.2854
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9295 - loss: 0.1756 - val_accuracy: 0.9234 - val_loss: 0.1746
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9621 - loss: 0.1227 - val_accuracy: 0.9324 - val_loss: 0.1789
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9607 - loss: 0.1318 - val_accuracy: 0.9324 - val_loss: 0.1953
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.9516 - loss: 0.1162 - val_accuracy: 0.7973 - val_loss: 0.8757
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.8643 - loss: 0.4216 - val_accuracy: 0.9369 - val_loss:

In [34]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_accuracy',
    save_best_only=True
)


In [36]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stop, checkpoint]
)


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 992ms/step - accuracy: 0.9954 - loss: 0.0145 - val_accuracy: 0.9505 - val_loss: 0.2912
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.9891 - loss: 0.0315 - val_accuracy: 0.9279 - val_loss: 0.4095
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9970 - loss: 0.0175 - val_accuracy: 0.9279 - val_loss: 0.4584
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 959ms/step - accuracy: 0.9799 - loss: 0.0603

25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9803 - loss: 0.0596 - val_accuracy: 0.9595 - val_loss: 0.3448


In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    # shear_range=15,
)


# datagen = ImageDataGenerator(
#     rotation_range=25,    # Slightly increased, but not too much
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     shear_range=0.1  # Just a little shear (not too aggressive)
# )


# datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     zoom_range=0.2,
#     shear_range=0.15,   # Skewing effect for robustness
#     brightness_range=[0.8, 1.2],  # simulates lighting variation
# )

In [ ]:
val_test_datagen = ImageDataGenerator()

# Final datasets
train_generator = datagen.flow(X_train, y_train, batch_size=32, shuffle=True)
val_generator = val_test_datagen.flow(X_val, y_val, batch_size=32, shuffle=False)
test_generator = val_test_datagen.flow(X_test, y_test, batch_size=32, shuffle=False)

# Step 4: Modelling:

### settings

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.layers import LeakyReLU, BatchNormalization, GlobalAveragePooling2D, Dropout, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
tf.test.is_gpu_available()

#### Model Evaluation

In [ ]:
# Function to Evaluate Keras Model
def evaluate_model(model, X_test, y_test, class_names):
    # Get model predictions
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)  # Convert probabilities to class labels
    y_true = np.argmax(y_test, axis=1)  # Convert one-hot encoded labels to class indices

    # Compute accuracy
    accuracy = accuracy_score(y_true, y_pred) * 100  # Convert to percentage

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Confusion Matrix")
    plt.show()

    # Print classification report
    print(classification_report(y_true, y_pred, target_names=class_names))
    print(f"Final Accuracy: {accuracy:.2f}%")  # Print final accuracy

## CNN Model

### Convolutional Neural Networks (CNN)
#### 🔹 Definition
A **Convolutional Neural Network (CNN)** is a deep learning model designed for image processing tasks. It automatically extracts features from images using convolutional layers.

#### 🔹 Key Components
- **Convolutional Layers**: Detect patterns using filters (kernels)
- **Pooling Layers**: Reduce dimensions while preserving features
- **Fully Connected Layers**: Classify the extracted features
- **Activation Functions**: Typically ReLU for non-linearity

#### 🔹 Advantages
✅ Captures spatial relationships in images  
✅ Reduces number of parameters compared to traditional networks  
✅ Suitable for image classification, object detection, and segmentation  
✅ no manual feature extraction & engineering needed.  

## Model Architecture

In [ ]:
# Build improved CNN model
def build_model():
    model = models.Sequential([
        # Input(shape=(64, 64, 3)),
        Input(shape=(128, 128, 3)),

        layers.Conv2D(32, (3, 3), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        layers.MaxPooling2D((2, 2)),
        Dropout(0.25),

        layers.Conv2D(256, (3, 3), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        layers.MaxPooling2D((2, 2)),
        Dropout(0.25),

        layers.Conv2D(512, (3, 3), padding='same'),
        BatchNormalization(),
        LeakyReLU(alpha=0.1),
        layers.MaxPooling2D((2, 2)),
        Dropout(0.25),

        GlobalAveragePooling2D(),
        Dense(256),
        LeakyReLU(alpha=0.1),
        Dropout(0.4),
        Dense(len(categories), activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

### 🧠 **CNN** Model Architecture – Full Explanation

This document explains the architecture and logic behind the custom Convolutional Neural Network (CNN) model used to classify EuroSAT satellite images.

---

#### 📥 Input Layer
```python
Input(shape=(128, 128, 3))
```
- Defines the shape of the input images.
- 128×128 pixels with 3 RGB channels.
- Essential for compatibility with model layers.

---

#### 🔁 Convolutional Layers

Each Layer extracts increasingly abstract spatial features.

#### 🔹 Layer 1
```python
Conv2D(32, (3, 3), padding='same')
BatchNormalization()
LeakyReLU(alpha=0.1)
MaxPooling2D((2, 2))
```
- 32 filters for edge detection and textures.
- BatchNormalization stabilizes learning.
- LeakyReLU prevents dead neurons.
- MaxPooling reduces spatial size.

#### 🔹 Layer 2
```python
Conv2D(64, (3, 3), padding='same')
BatchNormalization()
LeakyReLU(alpha=0.1)
MaxPooling2D((2, 2))
```
- Deeper filters (64) capture more complex features.
- Follows same structure as Block 1.

#### 🔹 Layer 3
```python
Conv2D(128, (3, 3), padding='same')
BatchNormalization()
LeakyReLU(alpha=0.1)
MaxPooling2D((2, 2))
Dropout(0.25)
```
- Adds dropout for regularization.
- More filters for deeper representations.

#### 🔹 Layer 4
```python
Conv2D(256, (3, 3), padding='same')
BatchNormalization()
LeakyReLU(alpha=0.1)
MaxPooling2D((2, 2))
Dropout(0.25)
```
- 256 filters for high-level abstraction.
- Continues downsampling and regularizing.

#### 🔹 Layer 5
```python
Conv2D(512, (3, 3), padding='same')
BatchNormalization()
LeakyReLU(alpha=0.1)
MaxPooling2D((2, 2))
Dropout(0.25)
```
- 512 filters: top-level spatial features.
- Final convolutional layer before pooling.

---

#### 🌍 Feature Aggregation

```python
GlobalAveragePooling2D()
```
- Averages each feature map into a single number.
- Reduces dimensionality, avoids overfitting.

---

#### 🧠 Dense Layers (Classifier)

```python
Dense(256)
LeakyReLU(alpha=0.1)
Dropout(0.4)
Dense(len(categories), activation='softmax')
```
- Dense(256): learns patterns from feature maps.
- Dropout(0.4): prevents overfitting.
- Final Dense uses softmax for classification across 10 EuroSAT classes.

---

#### ⚙️ Compilation

```python
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
```
- Adam: adaptive optimizer for faster convergence.
- Categorical crossentropy: suitable for multi-class classification.
- Accuracy as the evaluation metric.

---

#### ⏱️ Callbacks

```python
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
```
- EarlyStopping halts training if val_loss stagnates for 5 epochs.
- ReduceLROnPlateau decreases learning rate for finer convergence.

---

## Model Training (Fitting)

### 64 resizing

In [ ]:
# model building
model_64 = build_model()

# Train the model
history = model_64.fit(
    train_generator,
    epochs=60,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
evaluate_model(model_64, test_generator, y_test, class_names=categories)

### 128 resizing

In [ ]:
# Create the model
model_128 = build_model()

#### 🏋️ Training Process

```python
history = model.fit(
    train_generator,
    epochs=60,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)
```
- Trains up to 60 epochs with validation monitoring.
- Uses generators for memory-efficient data loading.

---

In [ ]:
# Train the model
history = model_128.fit(
    train_generator,
    epochs=60,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
# Evaluate the model on test data
evaluate_model(model_128, test_generator, y_test, categories)

####  Final Accuracy


#### Plot Accuracy and Loss Curves

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

#### Model Saving

In [ ]:
model_128.save('better_cnn_model.h5')

## Import and Test Model

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model("best_cnn_model.h5", compile=False)

In [ ]:
evaluate_model(model, test_generator, y_test, categories)

### Test Prediction

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

def predict_image(img_path, model, class_names):
    # Step 1: Load image
    original_img = image.load_img(img_path)
    img = image.load_img(img_path, target_size=(128, 128))  # resize to match input

    # Step 2: Convert to array
    img_array = image.img_to_array(img)

    # Step 3: Normalize (scale to 0-1)
    img_array = img_array / 255.0

    # Step 4: Expand dimensions (make it batch-like)
    img_batch = np.expand_dims(img_array, axis=0)

    # Step 5: Predict
    prediction = model.predict(img_batch)
    class_index = np.argmax(prediction)

    # Step 6: Show result
    predicted_label = class_names[class_index]
    print(f"Predicted Class: {predicted_label} ({prediction[0][class_index]*100:.2f}%)")

    # Optional: Display the image
    plt.imshow(original_img)
    plt.title(f"Predicted: {predicted_label}")
    plt.axis('off')
    plt.show()

In [ ]:
predict_image("C:", model, categories)